In [22]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from mylib import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"
# Connect to database via CRUD Module
db = AnimalShelter(username, password,'nv-desktop-services.apporto.com', 31499, 'AAC', 'animals')



# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# Add a new column 'Rescue_Type' based on the provided table
df['Rescue_Type'] = pd.cut(
    df['age_upon_outcome_in_weeks'],
    bins=[0, 26, 156, 300],  # Updated bins to cover the entire range of ages
    labels=['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking'],
    right=False  # Set right to False to include the left bin edge and exclude the right bin edge
)


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        # Grazioso Salvare logo with a link to www.snhu.edu
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), id='logo-id',
                     style={'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}),
            href='https://www.snhu.edu'
        ),
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        # Unique identifier containing your name
        html.P("Created by: Zoe Domagalski", id='creator-id', style={'text-align': 'center'}),
    ]),
    html.Hr(),
    html.Div([
        # Interactive filter options
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'},
            ],
            value='reset'
        ),
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_current=0,
                         page_size=10,
                         sort_action='native',
                         filter_action='native',
                         style_table={'height': '500px', 'overflowY': 'auto'},
                         style_data_conditional=[{
                             'if': {'row_index': 'odd'},
                             'backgroundColor': 'rgb(248, 248, 248)'
                         }],
                         style_header={
                             'backgroundColor': 'rgb(230, 230, 230)',
                             'fontWeight': 'bold'
                         }
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type is None or filter_type == 'reset':
        # Reset to the original data when 'Reset' is selected or when the app starts
        return df.to_dict('records')
    else:
        # Filter the data based on the selected rescue type
        rescue_mapping = {
            'water': 'Water Rescue',
            'mountain': 'Mountain or Wilderness Rescue',
            'disaster': 'Disaster or Individual Tracking'
        }
        filter_type = rescue_mapping.get(filter_type, '')  # Get the corresponding Rescue_Type
        df_filtered = df[df['Rescue_Type'] == filter_type]
        return df_filtered.to_dict('records')
    
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if 'breed' column is present in dff
    if 'breed' not in dff.columns:
        return []
    
     # Calculate the top N breeds based on frequency
    top_n = 10  # Set your desired number of top breeds
    top_breeds = dff['breed'].value_counts().nlargest(top_n).index

    # Filter the data for the top breeds
    dff_filtered = dff[dff['breed'].isin(top_breeds)]

    # Update the pie chart based on the filtered data
    return [
        dcc.Graph(            
             figure=px.pie(
                dff_filtered,
                names='breed',
                title=f'Top {top_n} Preferred Breeds',
                hole=0.0,
                color_discrete_sequence=px.colors.sequential.RdBu,
                labels={'breed': 'Breed'},
                )
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [
        {
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns
    ]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):  
    if not viewData:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)

    # Set a default value for selected_row_index
    selected_row_index = 0

    # Check if a row is selected
    if selected_rows:
        selected_row_index = selected_rows[0]

    lat, long = dff.loc[selected_row_index, 'location_lat'], dff.loc[selected_row_index, 'location_long']

    return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, long], zoom=15, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[lat, long], children=[
            dl.Tooltip(dff['breed']),        
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.loc[selected_row_index, 'name'])
            ])
        ])
    ])]


if __name__ == '__main__':
    app.run_server(debug=True, mode="external", port=8050) 
    # getting an error with the sample code, so looked up a quick fix.

Dash app running on http://127.0.0.1:8050/
